In [36]:
import dask.bag as db
import dask.array as da
import dask.dataframe as df
import math
import pandas as pd
import numpy as np
import requests

In [37]:
from dask.distributed import Client
client = Client()  # set up local cluster on your laptop
client

D:\Software\Anaconda\lib\site-packages\distributed\node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 52216 instead
  http_address["port"], self.http_server.port


Client Scheduler: tcp://127.0.0.1:52217 Dashboard: http://127.0.0.1:52216/status,Cluster Workers: 4 Cores: 12 Memory: 16.90 GB


Helper function that displays all the functions that can be invoked on an object.

In [38]:
def display_object_method(obj):
    object_methods = [method_name for method_name in dir(obj)
                  if callable(getattr(obj, method_name))]
    print(object_methods)

In [39]:
def get_split_words(word_bag):    
    split_words = word_bag.str.split()
    stripped_words = split_words.flatten().map(lambda x: x.strip())
    word_array = stripped_words.filter(lambda x: x!= "")
    return word_array

In [40]:
def get_filtered_words(words):
    stopwords = open("data/stopwords.txt", "r").read().split("\n")
    return words.filter(lambda x: x not in stopwords)

In [41]:
def get_words_for_document(document):
    word_bag = db.read_text(document)
    split_words = get_split_words(word_bag)
    long_words = split_words.filter(lambda x: len(x) < 3)
#     words_without_punctuations = long_words.map(puntuation_stripper)
    return get_filtered_words(long_words)    
#     return long_words

In [42]:
def get_IDF(words_by_document):
    unique_words = []
    for words_for_single_document in words_by_document:
        unique_words.append(words_for_single_document.distinct())
    large_bag = db.concat(unique_words)
    frequencies = large_bag.frequencies()
    idf = frequencies.map(lambda x: (x[0], round(math.log((len(words_by_document) + 1)/x[1]), 10)))
    return idf

In [43]:
def get_TF_IDF(tf, idf):
    tf = np.asarray(tf.frequencies().compute())
    tf_idf = {}
    for idf_item in idf: 
        col_name = str(int(idf_item[0], 16))
        tf_idf[col_name] = idf_item[1]
        for i in range(len(tf)):
            tf_item = tf[i]
            if idf_item[0] == tf_item[0]:
                tf_idf[col_name] = float(idf_item[1]) * float(tf_item[1])
                np.delete(tf, i)
                break
    return tf_idf

In [44]:
set_name = "X_small_train.txt"
# set_name = "X_train.txt"
# url = "data/train/" + set_name
url = 'https://storage.googleapis.com/uga-dsp/project1/files/' + set_name
file_names = pd.read_csv(url, header=None)[0].to_numpy()

In [45]:
all_file_words = []
for file_name in file_names:
#     file_name = "data/train/" + file_name + ".bytes"
    file_name = "https://storage.googleapis.com/uga-dsp/project1/data/bytes/" + file_name + ".bytes"
    
    all_file_words.append(get_words_for_document(file_name))
idf = get_IDF(all_file_words)

distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


In [46]:
idf = np.asarray(idf.topk(256, key=0).compute())

distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


In [47]:
cols = np.arange(255, -1, -1).astype(str)
pd_df = pd.DataFrame(columns=cols)

for file_words in all_file_words:
    tf_idf_bag_array = get_TF_IDF(file_words, idf)
    pd_df = pd_df.append(tf_idf_bag_array, ignore_index=True)

distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


,255,254,253,252,251,250,249,248,247,246,...,9,8,7,6,5,4,3,2,1,0
0,6.758897,10.459127,10.332478,10.828521,10.269153,10.812690,16.018578,11.419552,10.844352,10.480235,...,10.453850,5.309621,10.485512,5.093547,10.000023,10.174166,10.406357,10.664932,11.403720,19.017139
1,18.271420,16.580513,17.229591,16.179457,16.664946,16.949907,25.989568,16.068639,16.379985,16.459141,...,17.451228,8.656131,17.229591,8.769438,16.770487,17.873392,17.625371,17.055449,29.187403,29.942046
2,0.002635,0.0052770571,0.0052770571,0.0052770571,0.0052770571,0.0052770571,0.0079260653,0.0052770571,0.0052770571,0.0052770571,...,0.0052770571,0.002635,0.0052770571,0.002635,0.0052770571,0.0052770571,0.0052770571,0.0052770571,0.0052770571,0.023715
3,1.836628,0.153035,0.100264,0.216359,0.084433,0.068602,0.126817,0.073879,0.068602,0.147758,...,0.026385,0.245059,0.116095,0.021080,0.137203,0.174143,0.184697,0.174143,0.422165,13.707518
4,13.383407,3.055416,2.274412,3.709771,2.200533,2.364122,3.598434,4.907663,3.092355,2.992091,...,2.881273,4.158105,3.266498,1.794467,3.620061,11.857547,9.166248,7.641179,16.839089,128.268847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,17.694345,16.965739,16.131964,16.612176,17.139881,16.807427,25.268296,16.744102,16.701886,17.029063,...,17.076557,8.650861,17.271808,8.334656,16.749379,17.277085,17.477613,17.102942,28.701914,28.152849
375,13.478268,16.100301,16.184734,16.226951,16.327215,16.416925,25.220740,15.947267,16.211119,16.147795,...,16.390539,8.421612,16.575236,8.168648,16.284998,16.174180,17.182098,16.411648,22.200579,22.121225
376,153.546859,47.245492,1.118736,62.026529,0.791559,1.102905,2.647306,60.628109,2.944598,1.899741,...,1.567286,28.231900,56.823351,30.231901,54.237593,65.108330,57.873485,2903.558189,83.187528,123.528396
377,9.111995,3.129295,2.928767,4.707135,3.984178,3.672832,5.524468,4.897109,2.543542,2.269135,...,0.754619,0.511199,1.108182,0.424243,1.556732,1.292879,1.883909,1.197892,3.635892,67.897272


In [52]:
csv_name = "data/" + set_name + ".csv"
pd_df.to_csv(csv_name)

In [50]:
tf_idf_df = df.from_pandas(pd_df, npartitions=1)
final_dataframe = tf_idf_df.compute()

In [51]:
final_dataframe

,255,254,253,252,251,250,249,248,247,246,...,9,8,7,6,5,4,3,2,1,0
0,6.758897,10.459127,10.332478,10.828521,10.269153,10.812690,16.018578,11.419552,10.844352,10.480235,...,10.453850,5.309621,10.485512,5.093547,10.000023,10.174166,10.406357,10.664932,11.403720,19.017139
1,18.271420,16.580513,17.229591,16.179457,16.664946,16.949907,25.989568,16.068639,16.379985,16.459141,...,17.451228,8.656131,17.229591,8.769438,16.770487,17.873392,17.625371,17.055449,29.187403,29.942046
2,0.002635,0.0052770571,0.0052770571,0.0052770571,0.0052770571,0.0052770571,0.0079260653,0.0052770571,0.0052770571,0.0052770571,...,0.0052770571,0.002635,0.0052770571,0.002635,0.0052770571,0.0052770571,0.0052770571,0.0052770571,0.0052770571,0.023715
3,1.836628,0.153035,0.100264,0.216359,0.084433,0.068602,0.126817,0.073879,0.068602,0.147758,...,0.026385,0.245059,0.116095,0.021080,0.137203,0.174143,0.184697,0.174143,0.422165,13.707518
4,13.383407,3.055416,2.274412,3.709771,2.200533,2.364122,3.598434,4.907663,3.092355,2.992091,...,2.881273,4.158105,3.266498,1.794467,3.620061,11.857547,9.166248,7.641179,16.839089,128.268847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,17.694345,16.965739,16.131964,16.612176,17.139881,16.807427,25.268296,16.744102,16.701886,17.029063,...,17.076557,8.650861,17.271808,8.334656,16.749379,17.277085,17.477613,17.102942,28.701914,28.152849
375,13.478268,16.100301,16.184734,16.226951,16.327215,16.416925,25.220740,15.947267,16.211119,16.147795,...,16.390539,8.421612,16.575236,8.168648,16.284998,16.174180,17.182098,16.411648,22.200579,22.121225
376,153.546859,47.245492,1.118736,62.026529,0.791559,1.102905,2.647306,60.628109,2.944598,1.899741,...,1.567286,28.231900,56.823351,30.231901,54.237593,65.108330,57.873485,2903.558189,83.187528,123.528396
377,9.111995,3.129295,2.928767,4.707135,3.984178,3.672832,5.524468,4.897109,2.543542,2.269135,...,0.754619,0.511199,1.108182,0.424243,1.556732,1.292879,1.883909,1.197892,3.635892,67.897272
